# Введение
●	target_sum – прогнозируемая переменная (объем оборота POS-транзакций клиента в последующие 3 месяца). Для одного клиента cl_id строим один прогноз.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [18]:
#ser_ts_sum = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).sum().astype('int64')
#ser_pos_count = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_POS'}).astype('int64')
#ser_notpos_count = train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_notPOS'}).astype('int64')
#train = train.drop(axis=1, labels='target_sum').join(ser_ts_sum, on=['cl_id', 'year', 'month'], how='left').join(ser_pos_count, on=['cl_id', 'year', 'month'], how='left').join(ser_notpos_count, on=['cl_id', 'year', 'month'], how='left')


'''
[description]
InsertDataFrame - pandas dataframe
ColumnName - name of transforming column
'''
def MakeDateYearMonth(InsertDataFrame, ColumnName):
    df = InsertDataFrame.copy()
    df[ColumnName] = pd.to_datetime(df[ColumnName], dayfirst=True)
    df['year'] = df[ColumnName].dt.year
    df['month'] = df[ColumnName].dt.month
    return df    

'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbysum(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    sdtype = df[TargetColumn].dtype
    ser = df.groupby(by=GroupByList).sum().rename(columns={TargetColumn:TargetName}).astype(sdtype)
    return ser
    
'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbycount(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    ser = df.groupby(by=GroupByList).count().rename(columns={TargetColumn:TargetName}).astype('int64')
    return ser    

'''
[description]
InsertDataFrame - pandas dataframe
SeriesList - list of pandas series
OnJoin - string or list of column name(s)
'''
def DfLeftJoinSer(InsertDataFrame, SeriesList, OnJoin):
    df = InsertDataFrame.copy()
    for i in SeriesList:
        df = df.join(i, on=OnJoin, how='left')
    return df

In [13]:
url_train = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/train.csv'
url_test = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/test.csv'

#train = pd.read_csv(url_train)
#test = pd.read_csv(url_test)

train = pd.read_csv(r'C:/Course/xsolla/Git/Machine-learning-course-task/train.csv')
test = pd.read_csv(r'C:/Course\xsolla/Git/Machine-learning-course-task/test.csv')

In [14]:
train2 = train.copy()

In [65]:
train = train2.copy()
train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0.0


In [17]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 11 columns):
PERIOD          490513 non-null datetime64[ns]
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_sum      490513 non-null float64
year            490513 non-null int64
month           490513 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(5), object(3)
memory usage: 41.2+ MB


# Метрики для модели
год, месяц - сколько берём? надо смотреть сколько есть  
количество POS-транзакций  
количество не-POS-транзакций  
сумма POS-транзакций  
сумма не- POS-транзакций

In [66]:
# Входные сырые данные + колонки
train = MakeDateYearMonth(train, 'PERIOD')
group_list = ['cl_id', 'year', 'month']
ser_ts_sum = train[train[['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).first().astype('float64')
ser_pos_amount_sum = groupbysum(train[train['trx_category'] == 'POS'][['cl_id', 'amount', 'year', 'month']],
                       group_list,
                       'amount',
                       'sum_pos')
ser_notpos_amount_sum = groupbysum(train[train['trx_category'] != 'POS'][['cl_id', 'amount', 'year', 'month']],
                       group_list,
                       'amount',
                       'sum_notpos')
ser_pos_count = groupbycount(train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       group_list,
                       'target_sum',
                       'count_pos')
ser_notpos_count = groupbycount(train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       group_list,
                       'target_sum',
                       'count_notpos')

train = DfLeftJoinSer(train.drop(axis=1, labels=['PERIOD','MCC', 'channel_type', 'currency', 'TRDATETIME', 'amount', 'trx_category', 'target_sum']).drop_duplicates(),
                      [ser_ts_sum, ser_pos_amount_sum, ser_notpos_amount_sum, ser_pos_count, ser_notpos_count],
                      ['cl_id', 'year', 'month'])
train.head()

,cl_id,year,month,target_sum,sum_pos,sum_notpos,count_pos,count_notpos
0,0,2017,10,0.0,7054.00,56562.0,2.0,2.0
2,0,2017,12,0.0,767.00,NaN,1.0,NaN
5,1,2017,10,0.0,11555.00,NaN,24.0,NaN
9,1,2017,7,0.0,3930.26,50000.0,10.0,1.0
10,1,2017,9,0.0,103285.28,95000.0,60.0,2.0


In [130]:
# Через промедуточный датафрейм
# train_trannsfo
train_tf = pd.DataFrame(columns=['cl_id', 'target_sum'])
for i in train.cl_id.unique():
    train_tf = train_tf.append(train[train['cl_id'] == i][['cl_id', 'target_sum']].iloc[0])
    
for mon in range(1,train.groupby('cl_id').count().month.max()+1): # 14
    train_tf[f'_{mon}_sum_pos'] = np.NaN
    train_tf[f'_{mon}_sum_notpos'] = np.NaN
    train_tf[f'_{mon}_count_pos'] = np.NaN
    train_tf[f'_{mon}_count_notpos'] = np.NaN


#train = train.sort_values(by=['cl_id', 'year', 'month'], ascending=[True, False, False])    
    
train_tf.head()

,cl_id,target_sum,_1_sum_pos,_1_sum_notpos,_1_count_pos,_1_count_notpos,_2_sum_pos,_2_sum_notpos,_2_count_pos,_2_count_notpos,...,_12_count_pos,_12_count_notpos,_13_sum_pos,_13_sum_notpos,_13_count_pos,_13_count_notpos,_14_sum_pos,_14_sum_notpos,_14_count_pos,_14_count_notpos
2,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,5.0,326211.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,9.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,10.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
for ind in train_tf.index[:2]:
    train_tf.loc[ind, '_1_sum_pos'] = 1
train_tf.head()

,cl_id,target_sum,_1_sum_pos,_1_sum_notpos,_1_count_pos,_1_count_notpos,_2_sum_pos,_2_sum_notpos,_2_count_pos,_2_count_notpos,...,_12_count_pos,_12_count_notpos,_13_sum_pos,_13_sum_notpos,_13_count_pos,_13_count_notpos,_14_sum_pos,_14_sum_notpos,_14_count_pos,_14_count_notpos
2,0.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,5.0,326211.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,9.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,10.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
#train = train.sort_values(by=['cl_id', 'year', 'month'], ascending=[True, False, False]) 
for ind in train_tf.index[:5]:
    mon = 1
    temp_id = 
    for ji in train[train['cl_id'] == i].index:
        up_index = train[train['cl_id'] == i].index[0]
        print(ji, up_index)
        train.iloc[up_index][f'{mon}_sum_pos'] = train.iloc[ji]['sum_pos']
        train.iloc[up_index][f'{mon}_sum_notpos'] = train.iloc[ji]['sum_notpos']
        train.iloc[up_index][f'{mon}_count_pos'] = train.iloc[ji]['count_pos']
        train.iloc[up_index][f'{mon}_count_notpos'] = train.iloc[ji]['count_notpos']
        mon+=1
        
    train_tf.loc[ind,'_1_sum_pos'] = 1

train_tf.head(10)    

,cl_id,target_sum,_1_sum_pos,_1_sum_notpos,_1_couunt_pos,_1_count_notpos,_2_sum_pos,_2_sum_notpos,_2_couunt_pos,_2_count_notpos,...,_12_couunt_pos,_12_count_notpos,_13_sum_pos,_13_sum_notpos,_13_couunt_pos,_13_count_notpos,_14_sum_pos,_14_sum_notpos,_14_couunt_pos,_14_count_notpos
2,0.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,5.0,326211.75,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,9.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
290,10.0,0.00,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
758,11.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,14.0,187776.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1108,20.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1188,21.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1215,22.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# преобразуем в "последний месяц, предпоследний месяц, предпредпоследний месяц
train = train.sort_values(by=['cl_id', 'year', 'month'], ascending=[True, False, False])
train['1_sum_pos'] = np.NaN
for i in train.cl_id.unique()[0:2]:
    mon = 1
    for ji in train[train['cl_id'] == i].index:
        up_index = train[train['cl_id'] == i].index[0]
        print(ji, up_index)
        train.iloc[up_index][f'{mon}_sum_pos'] = train.iloc[ji]['sum_pos']
        train.iloc[up_index][f'{mon}_sum_notpos'] = train.iloc[ji]['sum_notpos']
        train.iloc[up_index][f'{mon}_count_pos'] = train.iloc[ji]['count_pos']
        train.iloc[up_index][f'{mon}_count_notpos'] = train.iloc[ji]['count_notpos']
        mon+=1
#train.drop(axis=1, labels=['year', 'month, 'sum_pos', 'sum_notpos', 'count_pos', ['count_notpos']).drop_duplicates() 
train

2 2


C:\Games\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Games\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DNS\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
C:\Games\Anaconda3\lib\site-packages\ip

0 2
5 5
10 5
11 5
9 5


,cl_id,year,month,target_sum,sum_pos,sum_notpos,count_pos,count_notpos,1_sum_pos
2,0,2017,12,0.00,767.00,NaN,1.0,NaN,NaN
0,0,2017,10,0.00,7054.00,56562.00,2.0,2.0,NaN
5,1,2017,10,0.00,11555.00,NaN,24.0,NaN,NaN
10,1,2017,9,0.00,103285.28,95000.00,60.0,2.0,NaN
11,1,2017,8,0.00,3807.50,NaN,7.0,NaN,NaN
9,1,2017,7,0.00,3930.26,50000.00,10.0,1.0,NaN
109,5,2017,6,9786352.50,90924.37,88878.38,30.0,7.0,NaN
114,5,2017,5,9133929.00,30119.90,47500.00,28.0,9.0,NaN
111,5,2017,4,12722258.25,117748.12,93500.00,39.0,12.0,NaN
129,5,2017,3,4566964.50,51708.36,25990.00,14.0,3.0,NaN


In [82]:
train_tf = pd.DataFrame(columns=['cl_id', 'target_sum'])
#tdf.columns = train.columns
#tdf.assign(M=np.NaN)
for i in train.cl_id.unique():
    train_tf = train_tf.append(train[train['cl_id'] == i][['cl_id', 'target_sum']].iloc[0])


# Анализ данных
какой месяц для модели текущий  
сколько для каждого клиента известно месяцев

In [25]:
# количество уникальных клиентов
print('number of cl_id', len(train.cl_id.unique()), end='\n\n')
print('count of months \n', train.groupby('cl_id').count().year.value_counts(), end='\n\n')
temp_df = train.groupby('cl_id').last()[['year', 'month', 'target_sum']]
temp_df['ym'] = temp_df.apply(lambda row: str(int(row.year))+'-'+str(int(row.month)), axis=1)
print('last month of target_sum > 1 \n', temp_df[temp_df['target_sum'] > 1].ym.value_counts(), end='\n\n' )
print('last month of target_sum < 1 \n', temp_df[temp_df['target_sum'] < 1].ym.value_counts(), end='\n\n' )

number of cl_id 5000

count of months 
 4     2894
3     1297
2      416
1      217
5      120
6       22
7       12
8        8
9        6
10       4
11       3
14       1
Name: year, dtype: int64

last month of target_sum > 1 
 2016-12    376
2017-7     334
2017-6     329
2017-4     306
2016-11    280
2017-5     274
2017-3     242
2017-2     231
2016-10    100
2017-1      84
2017-8      64
2017-10     38
2017-9      34
2017-11     27
2018-1      16
2018-2      14
2017-12     12
2018-3       6
Name: ym, dtype: int64

last month of target_sum < 1 
 2016-12    410
2016-11    303
2017-4     219
2017-5     203
2017-6     196
2017-2     192
2017-3     191
2017-7     185
2016-10    132
2017-1      69
2017-8      45
2017-9      16
2017-10     16
2017-12     15
2017-11     15
2018-2      11
2018-1       8
2018-3       7
Name: ym, dtype: int64



In [88]:
for mon in range(1,train.groupby('cl_id').count().month.max()+1): # 14
    train_tf.assign('1_sum_pos' = np.NaN)

SyntaxError: keyword can't be an expression (<ipython-input-88-eb6ffc5cccb4>, line 2)

In [176]:
train

,cl_id,year,month,target_sum,pos_sum,notpos_sum,count_pos,count_notpos
0,0,2017,1,0.00,7821.00,56562.00,3.0,2.0
5,1,2017,1,0.00,122578.04,145000.00,101.0,3.0
109,5,2017,1,36209504.25,290500.75,255868.38,111.0,31.0
251,9,2017,1,0.00,12365.09,836950.00,29.0,10.0
290,10,2017,1,0.00,200956.11,923387.88,374.0,89.0
753,11,2017,1,0.00,245747.14,182000.00,207.0,10.0
970,14,2017,1,24786532.32,258990.17,189700.00,132.0,4.0
1106,20,2017,1,0.00,222308.09,215000.00,74.0,3.0
1183,21,2016,1,0.00,144670.59,71000.00,78.0,6.0
1184,22,2017,1,0.00,110267.00,139500.00,45.0,14.0


In [112]:
asd = '_1_sum_pos'
mon=1
#train_tf[].assign(asd=np.NaN)
train_tf[f'_{mon}_sum_pos'] = np.NaN
train_tf

,cl_id,target_sum,_1_sum_pos
0,0.0,0.00,NaN
5,1.0,0.00,NaN
109,5.0,326211.75,NaN
251,9.0,0.00,NaN
290,10.0,0.00,NaN
753,11.0,0.00,NaN
970,14.0,187776.76,NaN
1106,20.0,0.00,NaN
1183,21.0,0.00,NaN
1184,22.0,0.00,NaN


In [191]:
'''for i in train.cl_id[10:30]:
    fig = plt.figure()
    x = list()
    y = list()
    temp_df = train[train['cl_id'] == i].reset_index(drop=True)
    for ji in temp_df.index:
        j = temp_df.iloc[ji]
        x.append(str(int(j.year))+ '-' + str(int(j.month)))
        y.append(j.pos_sum + j.notpos_sum)
    x.append('target')
    y.append(j.target_sum)
    print(x,y)
    plt.plot(x,y)
    plt.show()
plt.close('all')  
temp_df'''

"for i in train.cl_id[10:30]:\n    fig = plt.figure()\n    x = list()\n    y = list()\n    temp_df = train[train['cl_id'] == i].reset_index(drop=True)\n    for ji in temp_df.index:\n        j = temp_df.iloc[ji]\n        x.append(str(int(j.year))+ '-' + str(int(j.month)))\n        y.append(j.pos_sum + j.notpos_sum)\n    x.append('target')\n    y.append(j.target_sum)\n    print(x,y)\n    plt.plot(x,y)\n    plt.show()\nplt.close('all')  \ntemp_df"

In [85]:
train_tf

,cl_id,target_sum
0,0.0,0.00
5,1.0,0.00
109,5.0,326211.75
251,9.0,0.00
290,10.0,0.00
753,11.0,0.00
970,14.0,187776.76
1106,20.0,0.00
1183,21.0,0.00
1184,22.0,0.00


# Нормализация!

In [34]:
train.groupby('cl_id').count().month.max()

14

# Формирование выборки

In [63]:
# Пример формирования тестовой выборки. Возьмем 10 случайных пользвоателей.
users_sample = train['cl_id'].sample(10).values
user_sample_df = train[(train['cl_id'].isin(users_sample))][:]
user_sample_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,year,month,target_sum,count_POS,count_notPOS
48794,2017-01-04,901,5814,type2,810,29APR17:00:00:00,402.0,POS,2017,1,58147131.81,223.0,17.0
48795,2017-01-04,901,5261,type2,810,15APR17:00:00:00,28490.5,POS,2017,1,58147131.81,223.0,17.0
48799,2017-01-03,901,5921,type2,810,03MAR17:00:00:00,389.0,POS,2017,1,58147131.81,223.0,17.0
48803,2017-01-04,901,5814,type2,810,14APR17:00:00:00,99.0,POS,2017,1,58147131.81,223.0,17.0
48804,2017-01-04,901,5921,type2,810,20APR17:00:00:00,385.0,POS,2017,1,58147131.81,223.0,17.0


In [64]:
# Для данных пользователей необходимо спрогнозировать рассчитанную величину'target_sum'.
user_sample_df[['cl_id','target_sum']].drop_duplicates()

,cl_id,target_sum
48794,901,58147131.81
74524,1339,7079905.92
74617,1339,2876211.78
99591,1813,921600.00
134218,2414,2637756.90
152544,2715,48508409.67
261930,4977,0.00
352786,7082,0.00
359055,7213,0.00
404364,8274,151400.62
